In [34]:
import pandas as pd
import numpy as np
from typing import Tuple, Dict, Any
from datetime import datetime
import os


class Config:
    """Configuration constants for VAT and duty calculations."""

    VAT_RATES = {
        'DE': 0.19, 'PT': 0.23, 'ES': 0.21, 'IE': 0.23,
        'SE': 0.25, 'NL': 0.21, 'DK': 0.25, 'FI': 0.255,
        'IT': 0.22, 'AT': 0.20, 'BE': 0.21, 'EE': 0.22
    }

    NL_VAT_RATE = 0.21
    DEFAULT_COMMISSION_RATE = 0.2
    IE_COMMISSION_RATE = 0.3
    CONSIGNMENT_THRESHOLD = 150

    # Output directory for CSV reports
    OUTPUT_DIR = "reports"

    low_value_columns = ['MRN', 'Line Item Quantity Imported', 'Line Item Quantity Returned', 'Line Item Unit Price',
                         'Consignment Value', 'VAT Rate', 'Consignee Country']


class DataLayer:

    @staticmethod
    def load_data(csv_path: str) -> pd.DataFrame:
        df = pd.read_csv(csv_path)

        df = DataLayer.clean_data(df)  # clean data
        df = DataLayer.add_calculated_fields(df)  # add calculated fields

        low_value_df, high_value_df = DataLayer.separate_data(df,
                                                              Config.CONSIGNMENT_THRESHOLD)  # separate data based on consignment value to two dataframes

        return low_value_df, high_value_df

    @staticmethod
    def clean_data(df: pd.DataFrame) -> pd.DataFrame:
        df = df[~df['Consignee Country'].isin(['IC', 'CH'])]  # Exclude IC and CH countries
        df['MRN'] = df['MRN'].replace(['#N/A', 'N/A', 'NA', 'na', ''], pd.NA)  # standardize missing values
        df.loc[df['MRN'].isna(), 'MRN'] = df.loc[df['MRN'].isna(), 'Parcel ID']  # copy Parcel ID where MRN is missing
        return df

    @staticmethod
    def add_calculated_fields(
            df: pd.DataFrame) -> pd.DataFrame:  # we add Cosnignment Value ( sum of all parcels under the same MRN ) and VAT Rate
        # Calculate line item total value (quantity × unit price)
        df['Line Item Total Value'] = df['Line Item Quantity Imported'] * df['Line Item Unit Price']

        # Consignment Value: sum of all line item total values per MRN
        df['Consignment Value'] = df.groupby('MRN')['Line Item Total Value'].transform('sum')

        df['VAT Rate'] = df['Consignee Country'].map(Config.VAT_RATES)

        if df['VAT Rate'].isna().any():
            missing_countries = df[df['VAT Rate'].isna()]['Consignee Country'].unique()
            print(f"⚠️  WARNING: Missing VAT rates for countries: {missing_countries}")

        return df

    @staticmethod
    def separate_data(df: pd.DataFrame, threshold: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
        high_value_df = df[df['Consignment Value'] > threshold].copy()
        low_value_df = df[df['Consignment Value'] <= threshold].copy()
        return low_value_df, high_value_df







In [35]:
class LowValueProcessor:

    @staticmethod
    def process_low_value_data(df: pd.DataFrame) -> list[Any]:
        df = LowValueProcessor.clean_columns(df)  # keep only relevant columns for low value consignments

        vat_per_country = LowValueProcessor.calculate_vat_per_country(df)  # calculate VAT per country
        return_vat_per_country = LowValueProcessor.calculate_return_vat_per_country(
            df)  # calculate VAT refunds for returned items

        # calculate DR revenue from low value returns
        DR_revenue, DR_revenue_table = LowValueProcessor.calculate_dr_revenue_from_lw_returns(return_vat_per_country)
        PC_return = return_vat_per_country['Total VAT Refund'].sum() - DR_revenue  # total PC return after DR revenue
        total_import_vat_paid = vat_per_country['Total VAT to Pay'].sum()
        total_vat_needs_to_be_collected_from_returns = return_vat_per_country['Total VAT Refund'].sum()

        # Save reports to CSV files
        df.to_csv("low_value_consignments.csv", index=False)
        vat_per_country.to_csv("vat_per_country_report_low_value.csv", index=False)
        return_vat_per_country.to_csv("return_vat_per_country_report_low_value.csv", index=False)
        DR_revenue_table.to_csv("dr_revenue_from_low_value_returns.csv", index=False)

        return [DR_revenue, PC_return, total_vat_needs_to_be_collected_from_returns, total_import_vat_paid]

    @staticmethod
    def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
        return df[Config.low_value_columns]  # keep only relevant columns for low value consignments

    @staticmethod
    def calculate_vat_per_country(df: pd.DataFrame) -> pd.DataFrame:
        # First, get unique MRN records (to avoid counting same consignment multiple times)
        unique_consignments = df.drop_duplicates(subset=['MRN'])

        # Calculate VAT amount for each consignment
        unique_consignments['VAT Amount'] = unique_consignments['Consignment Value'] * unique_consignments['VAT Rate']

        # Group by Country and VAT Rate
        summary = unique_consignments.groupby(['Consignee Country', 'VAT Rate']).agg({
            'Consignment Value': 'sum',
            'VAT Amount': 'sum'
        }).reset_index()

        # Rename columns for clarity
        summary.columns = ['Country', 'VAT Rate', 'Total Consignment Value', 'Total VAT to Pay']

        return summary

    @staticmethod
    def calculate_return_vat_per_country(df: pd.DataFrame) -> pd.DataFrame:
        # Filter rows where items were returned
        returned_df = df[df['Line Item Quantity Returned'] > 0].copy()

        # Calculate total returned value for each line item
        returned_df['Returned Item Value'] = (
                returned_df['Line Item Quantity Returned'] *
                returned_df['Line Item Unit Price']
        )

        # Calculate VAT refund for each returned item
        returned_df['VAT Refund'] = returned_df['Returned Item Value'] * returned_df['VAT Rate']

        # Group by Country and VAT Rate
        summary = returned_df.groupby(['Consignee Country', 'VAT Rate']).agg({
            'Returned Item Value': 'sum',
            'VAT Refund': 'sum'
        }).reset_index()

        # Rename columns for clarity
        summary.columns = ['Country', 'VAT Rate', 'Total Returned Value', 'Total VAT Refund']

        return summary

    @staticmethod
    def calculate_dr_revenue_from_lw_returns(return_vat_per_country: pd.DataFrame) -> Tuple[float, pd.DataFrame]:
        # Create a copy to avoid modifying original
        revenue_df = return_vat_per_country.copy()

        # Calculate revenue: 30% for Ireland, 20% for others
        revenue_df['Revenue'] = revenue_df.apply(
            lambda row: row['Total VAT Refund'] * 0.30 if row['Country'] == 'IE'
            else row['Total VAT Refund'] * 0.20,
            axis=1
        )

        # Keep only Country and Revenue columns
        result_df = revenue_df[['Country', 'Revenue']]

        # Calculate total revenue
        total_revenue = result_df['Revenue'].sum()

        return total_revenue, result_df




In [36]:
def main():
    csv_path = "TED BAKER DUTY CLAIM BACK Jul-Sep v2.csv"

    print("📊 Starting VAT Analysis...")

    low_value_df, high_value_df = DataLayer.load_data(csv_path)

    low_values_data = LowValueProcessor.process_low_value_data(low_value_df)

    print(low_values_data)

    print("\n✅ Analysis complete!")

In [37]:
if __name__ == "__main__":
    main()

📊 Starting VAT Analysis...
[np.float64(280.189), np.float64(756.4359999999999), np.float64(1036.625), np.float64(9127.879500000001)]

✅ Analysis complete!


/var/folders/yz/6pjrzx695kzdch62s9tfzthh0000gn/T/ipykernel_50767/76116962.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MRN'] = df['MRN'].replace(['#N/A', 'N/A', 'NA', 'na', ''], pd.NA)  # standardize missing values
/var/folders/yz/6pjrzx695kzdch62s9tfzthh0000gn/T/ipykernel_50767/2729916500.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_consignments['VAT Amount'] = unique_consignments['Consignment Value'] * unique_consignments['VAT Rate']
